In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
data_dir = "/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/CD4_CD8_final_version/"

## check the patients I can use with enough cells in subtypes


In [2]:
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells_annotated.h5ad')

In [3]:
# get adata_CD4EX and adata_CD8EX
adata_CD4EX = adata_T[adata_T.obs['cell_type'] == 'CD4 EX', :]
adata_CD8EX = adata_T[adata_T.obs['cell_type'] == 'CD8 EX', :]

In [4]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD4EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD4EX = set(valid_patients_CD4EX)
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints in CD8EX
cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD8EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD8EX = set(valid_patients_CD8EX)

/tmp/ipykernel_3301121/2168074212.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()
/tmp/ipykernel_3301121/2168074212.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [5]:
adata_B = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_B_cells.h5ad')

In [ ]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_B.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_B = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_B = set(valid_patients_B)

/tmp/ipykernel_3301121/292830472.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_M1.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [ ]:
# filter adata_B using valid_patients_B
# adata_B = adata_B[adata_B.obs['patient_id'].isin(valid_patients_B), :]
adata_B.obs['expansion'].value_counts()

expansion
NE    4340
E     3903
Name: count, dtype: int64

In [ ]:
adata_B.obs['timepoint'].value_counts()

timepoint
on     4528
pre    4454
Name: count, dtype: int64

In [ ]:
# check the overlap of valid patients between CD4EX and M1
valid_patients_CD8EX & valid_patients_B

{'BIOKEY_10',
 'BIOKEY_11',
 'BIOKEY_12',
 'BIOKEY_13',
 'BIOKEY_15',
 'BIOKEY_16',
 'BIOKEY_28',
 'BIOKEY_31',
 'BIOKEY_4',
 'BIOKEY_5',
 'BIOKEY_8'}

# get the DEGs of filtered patients of CD8EX and Mono

In [ ]:
Patients_CD8EX_B = list(valid_patients_CD8EX & valid_patients_B)
adata_CD8EX_1 = adata_CD8EX[adata_CD8EX.obs['patient_id'].isin(Patients_CD8EX_B), :]
adata_B_1 = adata_B[adata_B.obs['patient_id'].isin(Patients_CD8EX_B), :]

In [12]:
DEG_1 = paird_ttest(adata_CD8EX_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_1 = DEG_1[DEG_1['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [13]:
# remove the DEGs with . in the gene names
DEG_1 = DEG_1[~DEG_1.index.str.contains('\.')]

In [ ]:
DEG_2 = paird_ttest(adata_B_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = DEG_2[DEG_2['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [15]:
DEG_2 = DEG_2[~DEG_2.index.str.contains('\.')]

## IVA

In [ ]:
adata_B_1

View of AnnData object with n_obs × n_vars = 5337 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_CD8EX_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_B_1, sample_id_col='sample_id')

In [ ]:
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_B' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')

In [20]:
# export the gene_df to csv
gene_df.to_csv(data_dir + 'gene_df_CD8EX_B.csv')

## load in IV result and perform CIT

In [21]:
DEG_pairs = pd.read_excel(data_dir + 'IV_regression_results_with_FisherZ_CD8EX_B.xlsx')

In [22]:
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})
calculate_gene_correlation(DEG_pairs, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.20784955601955354' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2


,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
0,ANKRD28_T,ACSM3_B,0.050323,-0.545864,0.007539,0.142714,0.047638,0.207850
1,ANKRD28_T,ALPK2_B,0.020877,0.189024,0.291734,0.140472,0.347388,0.544689
2,ANKRD28_T,AMN1_B,0.055209,-0.420274,0.019707,0.143803,0.071226,0.230498
3,ANKRD28_T,CAVIN1_B,0.032055,-0.332386,0.019737,0.140472,0.071226,-0.334084
4,ANKRD28_T,CCL4_B,0.040369,0.154740,0.385737,0.140472,0.430483,0.495582
...,...,...,...,...,...,...,...,...
2910,ZNF600_T,ZBTB10_B,0.101617,-1.237599,0.017184,0.157280,0.067873,-0.263169
2911,ZNF600_T,ZBTB11.AS1_B,0.206048,-0.864751,0.072140,0.223449,0.134113,0.061710
2912,ZNF600_T,ZNF346_B,0.103588,-0.932893,0.038003,0.159010,0.095664,-0.292602
2913,ZNF600_T,SIGLEC14_B,0.120261,-1.477682,0.011214,0.166840,0.055351,-0.173386


In [23]:
# Combine the two conditions using the logical OR operator
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.45) | (DEG_pairs['g1vsg2_correlation'] < -0.45)]
DEG_pairs = DEG_pairs[(DEG_pairs['p_value_ci'] > 0.1)]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
1893,OASL_T,IFIT1_B,0.037984,0.646853,0.560655,0.140472,0.593216,0.829875
787,DNAJB1_T,DNAJB1_B,0.000341,0.593584,0.180015,0.140472,0.243501,0.824649
127,ATF3_T,DNAJB1_B,0.000331,0.595953,0.350249,0.140472,0.399599,0.800751
589,CH25H_T,RASL11A_B,0.000484,0.599157,0.936904,0.140472,0.945009,0.786361
2790,ZFP36L1_T,RGCC_B,0.004389,0.549625,0.343722,0.140472,0.394390,0.766493
...,...,...,...,...,...,...,...,...
653,CISH_T,TPK1_B,0.021508,0.492596,0.708442,0.140472,0.734392,-0.727972
1037,FASLG_T,TNFRSF10D_B,0.001173,0.506222,0.827372,0.140472,0.843283,-0.728414
1831,NR4A2_T,CR2_B,0.001874,0.481046,0.340227,0.140472,0.391536,-0.737574
244,BCL2L11_T,IGHA1_B,0.008220,0.523082,0.960724,0.140472,0.967027,-0.738853


In [24]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')

In [25]:
# remove the ppi_prediction and ppi_prediction_go
lrpair = lrpair[lrpair['database'] != 'ppi_prediction']
lrpair = lrpair[lrpair['database'] != 'ppi_prediction_go']
lrpair = lrpair[['from', 'to']]
# change from as L and to as R
lrpair.columns = ['L', 'R']
lrpair

,L,R
0,CXCL1,CXCR2
1,CXCL2,CXCR2
2,CXCL3,CXCR2
3,CXCL5,CXCR2
4,PPBP,CXCR2
...,...,...
1387,COL2A1,SDC1
1388,LAMA1,SV2A
1389,COL2A1,GP6
1390,LAMA1,DAG1


In [26]:
# Check expression in adata_1_pseudo
expressed_genes_1 = adata_1_pseudo.var_names[adata_1_pseudo.X.sum(axis=0) > 0].tolist()

# Check expression in adata_2_pseudo
expressed_genes_2 = adata_2_pseudo.var_names[adata_2_pseudo.X.sum(axis=0) > 0].tolist()

# Filter lrpair where both L and R are expressed in their respective datasets
lrpair = lrpair[
    (lrpair['L'].isin(expressed_genes_1)) & 
    (lrpair['R'].isin(expressed_genes_2))
].reset_index(drop=True)
lrpair

,L,R
0,CXCL9,CXCR3
1,CXCL10,CXCR3
2,CXCL11,CXCR3
3,CX3CL1,CX3CR1
4,CCL19,CCR7
...,...,...
143,VCAM1,ITGB1
144,VEGFC,NRP2
145,WNT5A,FZD2
146,COL2A1,ITGA1


## CIT Fisherz

In [27]:
results = CIT_test(DEG_pairs, lrpair, adata_1_pseudo, adata_2_pseudo, gene_df, method = 'fisherz', p_value_threshold=0.05)

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib

In [28]:
#calculate the correlation between g1 and g2
calculate_gene_correlation(results, adata_1_pseudo, adata_2_pseudo)
calculate_residuals_correlation(results, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7145984536246673' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2
/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:1060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.02621419012844246' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.at[index, 'g1_residualvsg2_residuals_correlation'] = corr


,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,TXNIP_T,TNFRSF10D_B,TNFSF10,TNFRSF10D,0.911424,0.714598,-0.026214
1,IRF8_T,LYZ_B,MPZ,MPZ,0.056800,0.689451,0.421057
2,IRF8_T,LYZ_B,MMP9,LRP1,0.088486,0.689451,0.381236
3,RAB38_T,LYZ_B,IL1B,IL1R1,0.061982,0.684343,0.413578
4,RAB38_T,LYZ_B,TNC,ITGB6,0.128046,0.684343,0.344072
...,...,...,...,...,...,...,...
3036,ZFP36L1_T,TNFRSF10D_B,TNFSF10,TNFRSF10D,0.144003,-0.635337,-0.331375
3037,ZFP36L2_T,THY1_B,COL1A1,ITGB1,0.094531,-0.652603,-0.374877
3038,OASL_T,TBCK_B,LTA,TNFRSF1B,0.054238,-0.677306,-0.424945
3039,OASL_T,TBCK_B,FASLG,FAS,0.103342,-0.677306,-0.366112


In [31]:
results_filtered = results[(results['pValue'] > 0.1)]
results_filtered = results_filtered[(results_filtered['g1vsg2_correlation'] > 0.45)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] < 0.3)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] > -0.3)]
results_filtered 

,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,TXNIP_T,TNFRSF10D_B,TNFSF10,TNFRSF10D,0.911424,0.714598,-0.026214
24,TXNIP_T,CNR1_B,TNFSF10,TNFRSF10D,0.212569,0.612073,0.285638
33,OASL_T,CCR1_B,FASLG,FAS,0.296783,0.602026,0.241081
49,CISH_T,IFIT1_B,FASLG,FAS,0.662300,0.586897,0.102579
54,NFKBIA_T,AMN1_B,TNFSF10,TNFRSF10D,0.189137,0.581193,0.299990
...,...,...,...,...,...,...,...
1511,PELI1_T,TPK1_B,ITGAM,ICAM1,0.294327,0.453033,0.242262
1513,PELI1_T,TPK1_B,OCLN,OCLN,0.605517,0.453033,0.121138
1534,ERN1_T,LINC01857_B,F11R,JAM3,0.320853,0.451886,0.229805
1563,HSPA1A_T,HAPLN3_B,TNFSF10,TNFRSF10B,0.468166,0.451206,0.169347


In [32]:
results_filtered['L_R'] = results_filtered['L'] + '_' + results_filtered['R']
results_filtered['L_R'].nunique()

47

In [33]:
results_filtered.to_csv(data_dir + 'CIT_test_results_CD8EX_B_results.csv')

In [34]:
import pandas as pd
import numpy as np

# Sample data (assuming results_filtered is already available)
# Adjusting the script to use absolute mean and determine final sign based on majority sign

grouped_results = results_filtered.groupby(["L", "R"])

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Calculate mean correlation values using absolute mean
    g1vsg2_corr_mean = np.abs(group["g1vsg2_correlation"]).mean()
    g1_res_vs_g2_res_mean = np.abs(group["g1_residualvsg2_residuals_correlation"]).mean()
    p_value_mean = group["pValue"].mean()
    
    # Determine final sign based on majority sign of correlations
    sign_g1vsg2 = np.sign(group["g1vsg2_correlation"]).sum()
    sign_g1_res_vs_g2_res = np.sign(group["g1_residualvsg2_residuals_correlation"]).sum()
    
    final_g1vsg2_corr = g1vsg2_corr_mean * (1 if sign_g1vsg2 >= 0 else -1)
    final_g1_res_vs_g2_res = g1_res_vs_g2_res_mean * (1 if sign_g1_res_vs_g2_res >= 0 else -1)
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1
    if len(gem1) > min_gene_num:
        temp_data.append({
            "Ligand": ligand,
            "Receptor": receptor,
            "GEM1_elements": gem1,
            "GEM2_elements": gem2,
            "Num_genes_GEM1": len(gem1),
            "Num_genes_GEM2": len(gem2),
            "GEM1vsGEM2_corr(mean)": final_g1vsg2_corr,
            "GEM1_residualvsGEM2_residual_corr(mean)": final_g1_res_vs_g2_res,
            "average_significance": p_value_mean
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))
grouped_df

,Ligand,Receptor,GEM1_elements,GEM2_elements,Num_genes_GEM1,Num_genes_GEM2,GEM1vsGEM2_corr(mean),GEM1_residualvsGEM2_residual_corr(mean),average_significance
0,CCL3,CCR1,"[IER5L_T, IER5_T, ZFP36L1_T]",[CCR1_B],3,1,0.512651,0.063220,0.790113
1,CCL4,CCR1,"[IER5L_T, IER5_T, ZFP36L1_T]",[CCR1_B],3,1,0.512651,-0.083908,0.721241
2,CCL5,CCR1,"[ARID5B_T, CXCR4_T, FASLG_T, IER5L_T, IER5_T, ...","[CCR1_B, IFIT1_B, IGHA1_B, ZBTB10_B, ZNF346_B]",6,5,0.496487,0.224988,0.368135
3,F11R,JAM3,"[ARID5B_T, CH25H_T, ERN1_T, HSPA1A_T, HSPA1B_T]","[CCR7_B, CHI3L2_B, CNR2_B, HAPLN3_B, LINC01678...",5,9,0.480103,0.240737,0.303311
4,FASLG,FAS,"[ARID5B_T, BCL2L11_T, CISH_T, CXCR4_T, CYTOR_T...","[AMN1_B, CCR1_B, IFIT1_B, IGHA1_B, IGHA2_B, IG...",13,14,0.490392,0.225922,0.349696
5,FN1,SDC2,"[RASGEF1B_T, TSC22D2_T, ZNF600_T]","[EMP2_B, THY1_B]",3,2,0.501450,0.179474,0.466475
6,IFNL1,IFNLR1,"[ARID5B_T, BCL2L11_T, CH25H_T, ERN1_T]","[CHI3L2_B, HAPLN3_B, LINC01678_B, P2RY14_B]",4,4,0.472520,0.257880,0.266527
7,MPZ,MPZ,"[PELI1_T, PRDM1_T, RAB38_T]",[CCL4_B],3,1,0.469343,0.169302,0.502120
8,TNF,TNFRSF1B,"[BCL2L11_T, CISH_T, ERN1_T, IER2_T, NR4A2_T, O...","[FASN_B, HAPLN3_B, IGHA1_B, IGHA2_B, MCTP2_B, ...",9,6,0.473260,0.210169,0.394585
9,TNFSF10,TNFRSF10B,"[CH25H_T, ERN1_T, HSPA1A_T, HSPA1B_T, TNF_T]","[CNR2_B, HAPLN3_B, IGHA1_B, LINC01678_B, MIR15...",5,6,0.484892,0.251182,0.284991


In [35]:
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))

In [36]:
# save the grouped_df to excel
grouped_df.to_excel(data_dir + 'CIT_results_CD8EXvsB_groupedLR.xlsx')